In [90]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium


In [91]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [92]:
from bs4 import BeautifulSoup # BeautifulSoup is in bs4 package 
import requests
import pandas as pd
import numpy as np
import geocoder # import geocoder

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [93]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
content = requests.get(URL)

soup = BeautifulSoup(content.text, 'html.parser')

column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighbor_df = pd.DataFrame(columns=column_names)
postalCodes = [];
boroughs= [];
neighborhoods = [];
        

In [94]:
first_table = soup.select_one("table:nth-of-type(1)")
for tr in first_table.find_all('tr'):
    cols = tr.find_all('td') 
    for col in cols:
        coltext = col.get_text()
        #get first 3 letters in postalcode
        postalCode = coltext[0:4]
        postalCode = postalCode.lstrip('\n')
        #split by ( and take first part as borough
        borough = coltext[4:].split("(", 1)[0]
        if borough.strip() != "" and borough.strip() != "Not assigned":
            #take 2nd part of split as neighborhood
            neighborhood = coltext[4:].split("(", 1)[1]
            neighborhood = neighborhood.rstrip(')\n')
            neighborhood = neighborhood.rstrip(') ')
            if neighborhood == "Not assigned":
                neighborhood = borough
                 
            #add values to arrays
            postalCodes.append(postalCode) 
            boroughs.append(borough)
            neighborhoods.append(neighborhood)

In [95]:
for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbor_df = neighbor_df.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [96]:
neighborhood_name = neighbor_df.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbor_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbor_df.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Parkwoods are 43.75242000000003, -79.32924245299995.


In [97]:
#connect to foursquare
CLIENT_ID = 'I2DXO1LKWKNNX0HCKMDRKAAWW1SUFL0WUAE2F510CGZXG2DG' 
CLIENT_SECRET = 'AY4LSKOTFK0KFCOEQ42PJLVGL2IX2KJ2X3OBE0TUKJCSTFHB' 
VERSION = '20180604'
LIMIT = '30'
radius = '500' # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id='+ CLIENT_ID + '&client_secret=' + CLIENT_SECRET + '&v=' + VERSION + '&ll=' + str(neighborhood_latitude) + ',' +  str(neighborhood_longitude) + '&radius='+ radius + '&limit=' + LIMIT

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e811f6ff7706a001bf9a7ce'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.75692000450003,
    'lng': -79.32302427998279},
   'sw': {'lat': 43.74791999550003, 'lng': -79.33546062601711}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs

In [98]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [99]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [100]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
       
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbor_df['Latitude'],
                                 longitudes=neighbor_df['Longitude'])


Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills)North
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills)South(Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
The Danforth  East
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East 

In [102]:
# check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,2,2,2,2,2,2
Alderwood / Long Branch,2,2,2,2,2,2
Bathurst Manor / Wilson Heights / Downsview North,2,2,2,2,2,2
Bayview Village,2,2,2,2,2,2
Bedford Park / Lawrence Manor East,2,2,2,2,2,2
...,...,...,...,...,...,...
Willowdale)West,2,2,2,2,2,2
Woburn,2,2,2,2,2,2
Woodbine Heights,2,2,2,2,2,2


In [103]:

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 2 uniques categories.


In [104]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Food & Drink Shop,Park
0,Parkwoods,0,1
1,Parkwoods,1,0
2,Victoria Village,0,1
3,Victoria Village,1,0
4,Regent Park / Harbourfront,0,1


In [105]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Food & Drink Shop,Park
0,Agincourt,0.5,0.5
1,Alderwood / Long Branch,0.5,0.5
2,Bathurst Manor / Wilson Heights / Downsview North,0.5,0.5
3,Bayview Village,0.5,0.5
4,Bedford Park / Lawrence Manor East,0.5,0.5
...,...,...,...
98,Willowdale)West,0.5,0.5
99,Woburn,0.5,0.5
100,Woodbine Heights,0.5,0.5
101,York Mills / Silver Hills,0.5,0.5


In [106]:

# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    


----Agincourt----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Alderwood / Long Branch----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Bathurst Manor / Wilson Heights / Downsview North----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Bayview Village----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Bedford Park / Lawrence Manor East----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Berczy Park----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Birch Cliff / Cliffside West----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Brockton / Parkdale Village / Exhibition Place----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----CN Tower / King and Spadina / Railway Lands /

1               Park   0.5


----Runnymede / The Junction North----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Scarborough Village----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----St. James Town----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----St. James Town / Cabbagetown----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Steeles West / L'Amoreaux West----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Studio District----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park--

In [ ]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [110]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [111]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [112]:

toronto_merged = neighbor_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,0,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.730600,-79.313265,0,NaN,NaN,NaN
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650295,-79.359166,0,NaN,NaN,NaN
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.723270,-79.451286,0,NaN,NaN,NaN
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661150,-79.391715,0,NaN,NaN,NaN


In [31]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [113]:

# CLUSTER 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,North York,0,NaN,NaN,NaN
1,North York,0,NaN,NaN,NaN
2,Downtown Toronto,0,NaN,NaN,NaN
3,North York,0,NaN,NaN,NaN
4,Downtown Toronto,0,NaN,NaN,NaN
...,...,...,...,...,...
98,Etobicoke,0,NaN,NaN,NaN
99,Downtown Toronto,0,NaN,NaN,NaN
100,East TorontoBusiness reply mail Processing Cen...,0,NaN,NaN,NaN
101,Etobicoke,0,NaN,NaN,NaN
